In [2]:
!pip install -U langchain-community qdrant-client -q
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install gdown

In [4]:
!gdown '<your id>'

Downloading...
From: https://drive.google.com/uc?id=12LYpgxDT7kSZ4BP1TPM7L78sYkhMGVsE
To: /content/document.zip
100% 4.34M/4.34M [00:00<00:00, 158MB/s]


In [5]:
!unzip document.zip

Archive:  document.zip
  inflating: document/data.csv       
  inflating: document/json/265.json  
  inflating: document/json/264.json  
  inflating: document/json/252.json  
  inflating: document/json/262.json  
  inflating: document/json/261.json  
  inflating: document/json/255.json  
  inflating: document/json/249.json  
  inflating: document/json/236.json  
  inflating: document/json/266.json  
  inflating: document/json/237.json  
  inflating: document/json/251.json  
  inflating: document/json/243.json  
  inflating: document/json/250.json  
  inflating: document/json/248.json  
  inflating: document/json/238.json  
  inflating: document/json/228.json  
  inflating: document/json/246.json  
  inflating: document/json/258.json  
  inflating: document/json/257.json  
  inflating: document/json/263.json  
  inflating: document/json/254.json  
  inflating: document/json/260.json  
  inflating: document/json/253.json  
  inflating: document/json/239.json  
  inflating: document/json/

In [6]:
import pandas as pd
import json
import os
# df[df['ID']==boluat_id]['Văn bản'][0]
texts_law=[]
path='document/json'
df=pd.read_csv('document/data.csv')
all_file=sorted(os.listdir(path))
for file_ in all_file:
  boluat_id=int(file_.split('.')[0])
  boluat=str(df[df['ID'] == boluat_id]['Văn bản'].values[0])
  with open(os.path.join(path,file_),'r') as f:
    json_data=json.load(f)
    for item in json_data:
      for text in item['docitem-5']:
        try:
          save_text=text['docitem-5']+'\n'+text['docitem-11']
        except:
          save_text=text['docitem-5']
        list_save_text=save_text.split('.')
        list_save_text.insert(1, boluat)
        new_save_text='. '.join(list_save_text)
        texts_law.append(new_save_text)
        # with open('law.txt', 'a') as f:
        #   f.write(str(new_save_text) + '<split>')


In [5]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader,TextLoader

In [6]:
# Cài đặt thông tin truy cập vector database
QDRANT_API_KEY = ""
QDRANT_URL = ""
HUGGINGFACE_API_KEY= ""

In [7]:
!huggingface-cli login --token=HUGGINGFACE_API_KEY

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import torch

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentenceTransformer("minhquan6203/paraphrase-vietnamese-law").to(device)


encoded_texts = []
model.eval()
for it, text in enumerate(tqdm(texts_law)):
  vector=model.encode(text, convert_to_tensor=True)
  encoded_texts.append(vector.cpu())

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

100%|██████████| 18623/18623 [06:26<00:00, 48.20it/s]


In [9]:
# nhớ xóa database trước khi up lên lại
from qdrant_client import QdrantClient

# Initialize the Qdrant client
client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

# Delete the collection
client.delete_collection(collection_name="luatvn")

print("Vector database deleted successfully!")


Vector database deleted successfully!


In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct

client = QdrantClient(url=QDRANT_URL, prefer_grpc=False, api_key=QDRANT_API_KEY)

# Tạo collection nếu chưa có
client.recreate_collection(
    collection_name="luatvn",
    vectors_config={
        "size": len(encoded_texts[0]),
        "distance": "Cosine"  # hoặc "Euclidean" hoặc "Dot"
    }
)

# Chèn từng điểm dữ liệu vào collection
for it, (vector, text) in enumerate(zip(encoded_texts, texts_law)):
    point = PointStruct(
        id=it,
        vector=vector,
        payload={
            "metadata": {"source":"law.txt"},
            "page_content": text
        }
    )
    client.upsert(collection_name="luatvn", points=[point])

print("Tạo vector database thành công!")


<ipython-input-10-0d2ba8377dcc>:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [ ]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Qdrant

client = QdrantClient(url=QDRANT_URL, prefer_grpc=False, api_key=QDRANT_API_KEY)
embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name="minhquan6203/paraphrase-vietnamese-law",
    api_key=HUGGINGFACE_API_KEY,
    model_kwargs={'device': 'auto'}
)
db = Qdrant(client=client,embeddings=embeddings,collection_name='luatvn')

In [ ]:
query = "tội hiếp dâm"
found_docs = db.similarity_search(query,k=2)
found_docs